In [102]:
from ggplot import *

C:\ProgramData\Anaconda3\lib\site-packages\ggplot\utils.py:81: FutureWarning: pandas.tslib is deprecated and will be removed in a future version.
You can access Timestamp as pandas.Timestamp
  pd.tslib.Timestamp,
C:\ProgramData\Anaconda3\lib\site-packages\ggplot\stats\smoothers.py:4: FutureWarning: The pandas.lib module is deprecated and will be removed in a future version. These are private functions and can be accessed from pandas._libs.lib instead
  from pandas.lib import Timestamp
C:\ProgramData\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


# Import Data

In [115]:
"""
Shows basic usage of the Sheets API. Prints values from a Google Spreadsheet.
"""
from __future__ import print_function
from apiclient.discovery import build
from httplib2 import Http
from oauth2client import file, client, tools

# Setup the Sheets API
SCOPES = 'https://www.googleapis.com/auth/spreadsheets.readonly'
store = file.Storage('credentials.json')
creds = store.get()
if not creds or creds.invalid:
    flow = client.flow_from_clientsecrets('client_secret.json', SCOPES)
    creds = tools.run_flow(flow, store)
service = build('sheets', 'v4', http=creds.authorize(Http()))

# Call the Sheets API
SPREADSHEET_ID = '1QUCKVDlDFdJO_F4woYjcQ47cBJzVQf4gjPnQeOTq9mc'
RANGE_NAME = 'Sheet1!A4:F'
result = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET_ID,
                                             range=RANGE_NAME).execute()
values = result.get('values', [])
#print(values)


# Setting up Dataframe

In [116]:
import pandas as pd
import numpy as np

#print(len(values))
#df = pd.DataFrame(np.array(values).reshape(, columns = list("abcdefghi"))

val_arry = np.array(values)
#pint(val_arry)
investment_df = pd.DataFrame(val_arry.reshape(len(values),6), columns = [
                                                                        "tran_dt", 
                                                                        "investment_name",
                                                                        "company_monthly_contribution",
                                                                        "self_monthly_contribution",
                                                                        "begin_balance",
                                                                        "ending_balance"
                                                                        ]
                   )

#casting column types
investment_df['tran_dt'] = investment_df['tran_dt'].astype('datetime64[ns]')
investment_df['begin_balance'] = investment_df['begin_balance'].astype('float')
investment_df['ending_balance'] = investment_df['ending_balance'].astype('float')

#setting up helper columns
investment_df['tran_year']  = pd.DatetimeIndex(investment_df['tran_dt']).year
investment_df['tran_month'] = pd.DatetimeIndex(investment_df['tran_dt']).month
investment_df['m_ror'] = (investment_df['ending_balance'] - investment_df['begin_balance'])/investment_df['ending_balance']*100

#output the types
print(investment_df.dtypes)


tran_dt                         datetime64[ns]
investment_name                         object
company_monthly_contribution            object
self_monthly_contribution               object
begin_balance                          float64
ending_balance                         float64
tran_year                                int64
tran_month                               int64
m_ror                                  float64
dtype: object


# HeatMap of ROR on AgriBank 401K

In [117]:
#import monthly_returns_heatmap as mrh

#pvt_price_history =  price_history_df.pivot(index='Price Date',columns='Store Name',values = 'Item Price' ).fillna(0)




In [118]:
import plotly.plotly as py
import plotly.graph_objs as go

title = 'ROR over time between investments'

labels = ['AgriBank 401k']#, 'Newspaper', 'Internet', 'Radio']

colors = ['rgba(67,67,67,1)']#, 'rgba(115,115,115,1)', 'rgba(49,130,189, 1)', 'rgba(189,189,189,1)']

mode_size = [8]#, 8, 12, 8]

line_size = [2]#, 2, 4, 2]

x_data = [list(investment_df['tran_dt'])]

y_data = [list(investment_df['m_ror'])]

traces = []

for i in range(0, 1):
    traces.append(go.Scatter(
        x=x_data[i],
        y=y_data[i],
        mode='lines',
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
    ))

    traces.append(go.Scatter(
        x=[x_data[i][0], x_data[i][11]],
        y=[y_data[i][0], y_data[i][11]],
        mode='markers',
        marker=dict(color=colors[i], size=mode_size[i])
    ))
    
layout = go.Layout(
    xaxis=dict(
        showline=True,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        linewidth=2,
        autotick=False,
        ticks='outside',
        tickcolor='rgb(204, 204, 204)',
        tickwidth=2,
        ticklen=5,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        ),
    ),
    yaxis=dict(
        showgrid=False,
        zeroline=False,
        showline=False,
        showticklabels=False,
    ),
    autosize=False,
    margin=dict(
        autoexpand=False,
        l=100,
        r=20,
        t=110,
    ),
    showlegend=False,
)

annotations = []

# Adding labels
for y_trace, label, color in zip(y_data, labels, colors):
    # labeling the left_side of the plot
    annotations.append(dict(xref='paper', x=0.05, y=y_trace[0],
                                  xanchor='right', yanchor='middle',
                                  text=label + ' {}%'.format(y_trace[0]),
                                  font=dict(family='Arial',
                                            size=16,
                                            color=colors,),
                                  showarrow=False))
    # labeling the right_side of the plot
    annotations.append(dict(xref='paper', x=0.95, y=y_trace[11],
                                  xanchor='left', yanchor='middle',
                                  text='{}%'.format(y_trace[11]),
                                  font=dict(family='Arial',
                                            size=16,
                                            color=colors,),
                                  showarrow=False))
# Title
annotations.append(dict(xref='paper', yref='paper', x=0.0, y=1.05,
                              xanchor='left', yanchor='bottom',
                              text='Main Source for News',
                              font=dict(family='Arial',
                                        size=30,
                                        color='rgb(37,37,37)'),
                              showarrow=False))
# Source
annotations.append(dict(xref='paper', yref='paper', x=0.5, y=-0.1,
                              xanchor='center', yanchor='top',
                              text='Source: PewResearch Center & ' +
                                   'Storytelling with data',
                              font=dict(family='Arial',
                                        size=12,
                                        color='rgb(150,150,150)'),
                              showarrow=False))

layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
py.iplot(fig, filename='news-source')    
